In [1]:
%load_ext autoreload
%autoreload 2

In [14]:

from AutoLLM.interfaces.api_client import APIClient
from config import API_KEY

url = "https://api.studio.nebius.ai/v1/"

client = APIClient(url=url, api_key=API_KEY)
client.load_generation_config({"temperature": 1.0})
resp = client.chat_completion(
    [
        {
            "role": "user",
            "content": "What is the capital of France?",
        }
    ]
)
print(resp)

The capital of France is Paris.


In [ ]:
from AutoLLM.modules.prompt_mutator import PromptMutator, PromptMutatorSchema

pm = PromptMutator()